# [1164. Product Price at a Given Date](https://leetcode.com/problems/product-price-at-a-given-date/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Products

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |
+---------------+---------+</pre>
(product_id, change_date) is the primary key (combination of columns with unique values) of this table.
Each row of this table indicates that the price of some product was changed to a new price at some date.
 

Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
<pre>+------------+-----------+-------------+
| product_id | new_price | change_date |
+------------+-----------+-------------+
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |
+------------+-----------+-------------+</pre>
Output: 
<pre>+------------+-------+
| product_id | price |
+------------+-------+
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |
+------------+-------+</pre>

In [1]:
# Pandas Schema
import pandas as pd

data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'],
        [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]

# data = [ #test case 4 from leet code
#     [1, 20, '2019-08-17'],
#     [2, 50, '2019-08-18'],
#     [1, 30, '2019-08-15'],
#     [1, 35, '2019-08-16'],
#     [2, 65, '2019-08-17'],
#     [3, 20, '2019-08-18']
# ]

products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype(
    {'product_id': 'Int64', 'new_price': 'Int64', 'change_date': 'datetime64[ns]'})

In [2]:
# to spark schema

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

products_df = spark.createDataFrame(products)
products_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:29:12 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:29:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:29:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+---------+-------------------+
|product_id|new_price|change_date        |
+----------+---------+-------------------+
|1         |20       |2019-08-14 00:00:00|
|2         |50       |2019-08-14 00:00:00|
|1         |30       |2019-08-15 00:00:00|
|1         |35       |2019-08-16 00:00:00|
|2         |65       |2019-08-17 00:00:00|
|3         |20       |2019-08-18 00:00:00|
+----------+---------+-------------------+


In [3]:
# In spark dataframe

#only records which has change_date = 2019-08-16
pdf1 = products_df.filter(F.col('change_date')=='2019-08-16').select('product_id',F.col('new_price').alias('price')).distinct()
pdf1.show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
+----------+-----+


In [4]:
from pyspark.sql import Window


#only records which has only change_date < 2019-08-16
pdf2 = products_df.join(pdf1, on=['product_id'], how = 'anti')\
    .filter((F.col('change_date')<'2019-08-16'))\
    .withColumn('flag',F.row_number().over(Window.partitionBy('product_id').orderBy(F.col('change_date').desc())))\
    .filter((F.col('flag')==1))\
    .select('product_id',F.col('new_price').alias('price')).distinct()
pdf2.show()

+----------+-----+
|product_id|price|
+----------+-----+
|         2|   50|
+----------+-----+


In [5]:
#only records which has only change_date > 2019-08-16
pdf3 = products_df.join(pdf1, on=['product_id'], how = 'anti')\
    .join(pdf2, on=['product_id'], how = 'anti')\
    .withColumn('new_price',F.lit(10))\
    .select('product_id',F.col('new_price').alias('price')).distinct()
pdf3.show()

+----------+-----+
|product_id|price|
+----------+-----+
|         3|   10|
+----------+-----+


In [6]:
#now combining all
final_df = pdf1.unionAll(pdf2).unionAll(pdf3)
final_df.show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   50|
|         3|   10|
+----------+-----+


In [7]:
# In spark SQL

products_df.createOrReplaceTempView('products')

spark.sql('''
with pdf1 as (
-- #only records which has change_date = 2019-08-16
    select distinct product_id, new_price as price 
    from products
    where change_date = '2019-08-16'
), pdf2 as (
-- #only records which has only change_date < 2019-08-16
    select product_id, new_price as price
    from (
        select distinct
            products.product_id, products.new_price, products.change_date, 
            row_number() over (partition by products.product_id order by products.change_date desc) as cc
        from products
        left join pdf1
        on pdf1.product_id = products.product_id
        where change_date<'2019-08-16' and pdf1.product_id is null
    ) p1 where p1.cc = 1
), pdf3 as (
-- #only records which has only change_date > 2019-08-16
    select distinct products.product_id, 10 as price
    from products
    left join pdf1
    on pdf1.product_id = products.product_id
    left join pdf2
    on pdf2.product_id = products.product_id
    where pdf1.product_id is null and pdf2.product_id is null
)
select * from pdf1
union all
select * from pdf2
union all
select * from pdf3;
''').show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   50|
|         3|   10|
+----------+-----+


In [8]:
spark.stop()